In [1]:
import Orange
import random
import scipy
import numpy as np
import itertools
from matplotlib import pyplot as plt
import collections
from operator import itemgetter
%matplotlib inline

# Klasifikatorji v Orange-u

Poleg [sklearn-a](http://scikit-learn.org/) je [Orange](http://orange.biolab.si), vsaj tako mislimo na FRI :-), eno pomembnejših okolij za strojno učenje v Pythonu. Med sklearn-om in Orangeom je kar nekaj razlik, med pomembnejšimi pa je vsekakor ta, da se Orange bolj ukvarja z opisom podatkov ter stremi k simboličnem učenju, torej učenju, pri katerih je pomembno razumeti tudi strukturo problema in odkrite vzorce. O slednjem tu ne bo govora. Namen tega zapisa je namreč raje predstaviti osnovne koncepte Orange in pa to, kako lahko v Orangeu razvijem nov, uporaben klasifikator. Konkretno, naš cilj je razviti logistično regresijo in jo zapakirati v obliko, ki bo univerzalno uporabna v Orange-u. Tudi za klasifikacijske primere, ki niso samo binarni. A po vrsti: najprej k podatkom.

## Podatki

Preberemo podatke o [rožicah](https://en.wikipedia.org/wiki/Iris_flower_data_set). Kje sploh je vhodna datoteka? Instalacija Orange pride z nekaj primeri podatkov, med katerimi je tudi datoteka iris.tab. Lahko jo poiščeš nekje na tvojem disku, Orange pa jo med datotekami zna poiskati tudi sam. Spodaj preberemo in malo pobrskamo po podatkih.

In [2]:
iris = Orange.data.Table("iris")

In [3]:
iris

[[5.100, 3.500, 1.400, 0.200 | Iris-setosa],
 [4.900, 3.000, 1.400, 0.200 | Iris-setosa],
 [4.700, 3.200, 1.300, 0.200 | Iris-setosa],
 [4.600, 3.100, 1.500, 0.200 | Iris-setosa],
 [5.000, 3.600, 1.400, 0.200 | Iris-setosa],
 ...
]

In [4]:
iris.X[:5]

array([[ 5.1,  3.5,  1.4,  0.2],
       [ 4.9,  3. ,  1.4,  0.2],
       [ 4.7,  3.2,  1.3,  0.2],
       [ 4.6,  3.1,  1.5,  0.2],
       [ 5. ,  3.6,  1.4,  0.2]])

In [5]:
iris.Y[:5]

array([ 0.,  0.,  0.,  0.,  0.])

In [6]:
iris[90].get_class() in ["Iris-versicolor", "Iris-virginica"]

True

Naša logistična regresija zna v osnovi obravnavati samo binarne probleme. Kasneje jo bomo zapakirali v ovojnivo, ki bo znala ta algoritem uporabiti in ga razširiti na večrazredne primere. A si do takrat raje pripravimo binarne podatke, na katerih bomo preiskušali osnovno verzijo algoritma. Naš iris (podatki o rožicah, ki smo si jih ogledovali zgoraj) ima tri vrednosti razredov. Spodaj izberemo samo primere, ki pripadajo dvem izbranim razredom. Tu dodatno še spremenimo domeno, kjer trirazredno spremenljivko zamenjamo s to s samo dvemi razredi.

In [7]:
targets = ["Iris-versicolor", "Iris-virginica"]
flower = Orange.data.domain.DiscreteVariable("flower", values=targets)
domain = Orange.data.Domain(iris.domain.attributes, flower)
table = [d for d in iris if d.get_class() in targets]
random.shuffle(table)
data = Orange.data.Table(domain, table)

In [8]:
data[:3]

[[4.900, 2.400, 3.300, 1.000 | Iris-versicolor],
 [6.700, 3.000, 5.000, 1.700 | Iris-versicolor],
 [6.100, 3.000, 4.600, 1.400 | Iris-versicolor]
]

In [9]:
data.Y[:3], data.Y[-3:]

(array([ 0.,  0.,  0.]), array([ 0.,  1.,  0.]))

## Gradnja modelov in razvrščanje

V Orangeu so learnerji in so model. Learnerji so objekti, ki lahko sprejmejo podatke in zgradijo model (klasifikator). Modeli so objekti, ki sprejmejo atributni opis primera in znajo napovedati razre, ali še bolje, verjetnosti razrednih vrednosti. Primer spodaj uporablja učni algoritem klasifikacijskega drevesa. Najprej pridobimo instanco learnerja, temu damo podatke, zgradimo model, nato pa model kar izpišemo.

In [10]:
tree_learner = Orange.classification.SimpleTreeLearner()

In [11]:
tree = tree_learner(data)

In [12]:
print(tree.to_string())


petal width ([50.0, 50.0])
: <=1.75
   petal length ([49.0, 5.0])
   : <=5.35
      petal length ([49.0, 3.0])
      : <=4.95
         sepal length ([47.0, 1.0])
         : <=4.95 --> Iris-versicolor ([1.0, 1.0])
         : >4.95 --> Iris-versicolor ([46.0, 0.0])
      : >4.95
         petal width ([2.0, 2.0])
         : <=1.55 --> Iris-virginica ([0.0, 2.0])
         : >1.55 --> Iris-versicolor ([2.0, 0.0])
   : >5.35 --> Iris-virginica ([0.0, 2.0])
: >1.75
   petal length ([1.0, 45.0])
   : <=4.85 --> Iris-virginica ([1.0, 2.0])
   : >4.85 --> Iris-virginica ([0.0, 43.0])


Poiščimo verjetnosti za prvih pet primerov. Ker imamo dva razreda, bodo verjetnosti podane v matriki, za vsak primer (vrstica) in stolpec (Iris versicolor ali virginica).

In [13]:
ex = data[:5]

In [14]:
ex

[[4.900, 2.400, 3.300, 1.000 | Iris-versicolor],
 [6.700, 3.000, 5.000, 1.700 | Iris-versicolor],
 [6.100, 3.000, 4.600, 1.400 | Iris-versicolor],
 [5.800, 2.700, 4.100, 1.000 | Iris-versicolor],
 [5.200, 2.700, 3.900, 1.400 | Iris-versicolor]
]

In [15]:
tree(data, 1)[:5]

array([[ 0.5,  0.5],
       [ 1. ,  0. ],
       [ 1. ,  0. ],
       [ 1. ,  0. ],
       [ 1. ,  0. ]])

## Prečno preverjanje

Ena od pogostih uporab learnerjev, torej objektov, ki se znajo naučiti modelov iz podatkov, je, da jih lahko posredujemo funkcijam in razredom, ki uporabljajo learnerje in z njimi nekaj počno na podatkih. Tipičen primer je vrednotenje in primerjava klasifikacijskih metod. Spodaj je primer prečnega preverjanja in primerjave dveh metod strojnega učenja. Za oceno uspešnosti učenja smo uporabili klasifikacijsko točnost.

In [16]:
tree_learner = Orange.classification.SimpleTreeLearner()
rf_learner = Orange.classification.RandomForestLearner()
knn_learner = Orange.classification.KNNLearner()
res = Orange.evaluation.CrossValidation(data, [tree_learner, rf_learner, knn_learner])

In [17]:
Orange.evaluation.CA(res)

array([ 0.89,  0.91,  0.94])

## Logistična regresija na oranžen način

Zdaj, ko vemo, kako se obnašajo objekti, ki se učijo in objekti, ki znajo klasificirati, lahko na ta način implementiramo logistično regresijo. Uporabimo kodo iz prejšnjih predavanj, in vse skupaj zapakiramo v dva razreda, enega za učenje in drugega za klasifikacijo.

In [18]:
def sigmoid(z):
    return 1. / (1 + np.exp(-z))

class LogRegLearner(Orange.classification.Learner):
    """Wraps logistic regression into an Orange learner"""
    def __init__(self):
        super().__init__()
        self.name = "logreg"
        self.X, self.y = None, None
        
    def J(self, theta):
        yh = sigmoid(self.X.dot(theta))
        return -sum(self.y*np.log(yh) + (1-self.y)*np.log(1-yh))

    def dJ(self, theta):
        return -(self.y - sigmoid(self.X.dot(theta))).dot(self.X)

    def optimize(self):
        res = scipy.optimize.fmin_l_bfgs_b(self.J, np.zeros(self.X.shape[1]).T, self.dJ)
        return res[0]

    def fit_storage(self, data):
        self.X = np.column_stack((np.ones(len(data)), data.X))
        self.y = data.Y
        theta = self.optimize()
        return LogRegModel(data.domain, theta)
        
class LogRegModel(Orange.classification.Model):
    def __init__(self, domain, theta):
        super().__init__(domain)
        self.theta = theta

    def predict_storage(self, data, ret=1):
        """Given a data instance or table of data instances returns predicted class."""
        X = data.X
        X1 = np.column_stack((np.ones(len(X)), X))
        y_hat = sigmoid(X1.dot(self.theta))
        res = np.column_stack((1-y_hat, y_hat))
        return res

Dela? Poskusimo.

In [19]:
lrl = LogRegLearner()

In [20]:
lr = lrl(data)

In [21]:
lr(data[0:10], 2)

(array([0, 0, 0, 0, 0, 0, 1, 1, 1, 0]),
 array([[  9.99999999e-01,   5.35170986e-10],
        [  7.23938847e-01,   2.76061153e-01],
        [  9.99840382e-01,   1.59617676e-04],
        [  9.99999985e-01,   1.48109667e-08],
        [  9.99985190e-01,   1.48103831e-05],
        [  9.98801398e-01,   1.19860220e-03],
        [  4.92776975e-08,   9.99999951e-01],
        [  3.86083735e-04,   9.99613916e-01],
        [  4.50208781e-09,   9.99999995e-01],
        [  9.99959763e-01,   4.02370316e-05]]))

In [22]:
data.Y[:10]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  0.])

Zgleda, da dela. Preiskusimo vse skupaj še s prečnim preverjanjem

In [23]:
titanic = Orange.data.Table("titanic")

logreg = LogRegLearner()
tree = Orange.classification.SimpleTreeLearner()
rf = Orange.classification.RandomForestLearner()

In [24]:
learners = [logreg, tree, rf]
res = Orange.evaluation.CrossValidation(titanic, learners, k=5)
Orange.evaluation.CA(res)

array([ 0.7760109 ,  0.78736938,  0.78736938])

Uf, sploh ni slabo. Kaj bi šele bilo, če bi vključili še kakšno regularizacijo, ali pa še kakšno drugo finto.

## Izbor parametra metode

Točnost napovednih modelov je odvisna od nastavitev (parametrov), ki jih ti modeli uporabljajo pri učenju. Na primer, pri regularizirani logistični regresije bo točnost na testnih podatkih prav gotovo odvisna od stopnje regularizacije. To stopnjo bi bilo dobro nastaviti na vrednost, pri kateri pričakujemo, da bo točnost metode na novih podatkih največja. Spodaj pokažemo, kako se spreminja točnost v odvisnosti od nastavitve parametra metode (gozd naključnih dreves, parameter je število dreves v ansamblu).

In [25]:
ns = [1, 5, 10, 20, 50, 100]
learners = [Orange.classification.RandomForestLearner(n_estimators=n) for n in ns]
data = Orange.data.Table("heart_disease")
res = Orange.evaluation.CrossValidation(data, learners, k=5)

In [26]:
scores = Orange.evaluation.AUC(res)
scores

array([ 0.71075788,  0.78040374,  0.8183637 ,  0.79093915,  0.81570767,
        0.8161541 ])

Kateri je potem najbolj primeren parameter?

In [27]:
max(zip(scores, ns))

(0.81836369648869645, 10)

# Ovojnica za učenje z oceno parametra metode

Napišimo sedaj razred, ki sprejme metodo učenja (learner), ime parametra, katerega optimalno vrednost glede na izbrano metodo ocenjevanja točnosti bi radi izbrali, in možne vrednosti tega parametra, ki bi jih radi ovrednotili. Vse skupaj torej zavijemo v ovojnico, ki sedaj predstavlja nov učni algoritem, torej tak, ki poišče ustrezno vrednost parametra in s to vrednostjo potem na celotnih učnih podatkih zgradi napovedni model. Iskanje prave vrednosti parametra je torej postal del učne metode.

In [28]:
def f(a=1, b=2):
    print(a, b)

z = {"a": 10}
f(**z)

10 2


In [29]:
class GuessParameterLearner(Orange.classification.Learner):
    """Wraps logistic regression into an Orange learner"""
    def __init__(self, learner, param_name, values):
        super().__init__()
        self.learner = learner
        self.param_name = param_name
        self.values = values
        
    def fit_storage(self, data):
        # internal cross-validation on training data first
        learners = [self.learner(**{self.param_name: v}) for v in self.values]
        res = Orange.evaluation.CrossValidation(data, learners, k=3)
        scores = Orange.evaluation.AUC(res)
        v_star = max(zip(scores, self.values))[1]
        learner = self.learner(**{self.param_name: v_star})
        model = learner(data)
        return GuessParameterClassifier(model.domain, model, v_star)
    
class GuessParameterClassifier(Orange.classification.Model):
    def __init__(self, domain, model, v):
        super().__init__(domain)
        self.model = model
        self.v_star = v

    def predict_storage(self, data, ret=1):
        """Given a data instance or table of data instances returns predicted class."""
        return self.model(data, 1)

Uporabimo sedaj zgornje, v novi razred zavijemo metodo gradnje naključnih dreves, in preverimo, če vse skupaj deluje v Orange-u v sklopu prečnega preverjanja.

In [30]:
learner = GuessParameterLearner(Orange.classification.RandomForestLearner, "n_estimators", [1, 10, 20])

In [31]:
learner(data)

learner

In [32]:
res = Orange.evaluation.CrossValidation(data, [learner], k=5, store_models=True)

In [33]:
Orange.evaluation.AUC(res)

array([ 0.81443603])

In [34]:
a = res.models[3][0]

In [35]:
a.v_star

10